In [1]:
import numpy as np
import math
from scipy import integrate
import matplotlib.pyplot as plt
import random
import scipy.stats
import time

In [2]:
def chi2(n):
    a = 2*n[0] + n[1]; b = 2*n[3] + n[4]
    R = 2*(n[0]+n[1]+n[2]); S = 2*(n[3]+n[4]+n[5])
    N = R+S
    
    return (2*N*((a*S-b*R)**2))/(R*S*(a+b)*(2*N-a-b))

In [3]:
# pseudo_SHD

def pseudo_SHD(n):
    a = 2*n[0] + n[1]; b = 2*n[3] + n[4]
    c = n[1] + 2*n[2]; d = n[4] + 2*n[5]
    
    return (math.fabs(a-b)+math.fabs(c-d))/4

In [4]:
# exponential mechanism

def random_num(pd):
    dist = np.cumsum(pd).tolist()
    dist[-1] = 1.0
    num = np.random.rand()
    dist.append(num)
    return sorted(dist).index(num)

def exp_mec(score, m, K, epsilon):
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    s = np.zeros(m)
    for i in range(m):
        s[i] = score[i]
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp(epsilon * s[i] / (2*K))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
            
        x = random_num(p)
        s[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [5]:
# (Efficient) Joint Permute-and-Flip

def JointPnF(score, m, K, epsilon):
    S = np.zeros(K)
    SS = np.zeros(K)
    w = np.zeros(m)
    
    si = np.argsort(-score)
    c = np.zeros(m)
    for i in range(m):
        c[si[i]] = i
    ss = sorted(score, reverse=True)
    for i in range(K-1,m):
        if i == K-1:
            C = 1
        else:
            C = (C/(i-K+1)) * i
        r = np.random.rand()
        if r**(1/C)==1:
            D = C
            p = 16
            while(1):
                D /= 10
                p += 1
                if r**(1/D) != 1:
                    l = (1 - r**(1/D))*(10**16)
                    break
            x = -(2/epsilon)*(math.log(l) + (-1*p)*math.log(10))
        else:
            x = -(2/epsilon)*math.log(1-r**(1/C))
        w[i] = ss[i] + x
    SS[K-1] = np.argmax(w[K-1:m]) + K-1
    SS[0:K-1] = np.random.choice(int(SS[K-1]), size=K-1, replace=False)
    
    for i in range(K):
        S[i] = c[int(SS[i])]
    
    return S

In [6]:
def generate_data(N, M):
    S = np.zeros(M)
    b = np.zeros(M)
    c = np.zeros(M)
    n = np.zeros((M,6))
    
    for i in range(M-10):
        n[i][0] = np.random.binomial(int(N/2),1/3)
        n[i][1] = np.random.binomial((int(N/2)-n[i][0]), 1/2)
        n[i][2] = int(N/2)-n[i][0]-n[i][1]
        n[i][3] = np.random.binomial(int(N/2),1/3)
        n[i][4] = np.random.binomial((int(N/2)-n[i][3]), 1/2)
        n[i][5] = int(N/2)-n[i][3]-n[i][4]
    
    for i in range(M-10,M):
        n[i][0] = np.random.binomial(int(N/2),1/7)
        n[i][1] = np.random.binomial((int(N/2)-n[i][0]), 1/4)
        n[i][2] = int(N/2)-n[i][0]-n[i][1]
        n[i][3] = np.random.binomial(int(N/2),1/2)
        n[i][4] = np.random.binomial((int(N/2)-n[i][3]), 1/1.5)
        n[i][5] = int(N/2)-n[i][3]-n[i][4]
    
    return n

In [7]:
def RunTime(M):
    epsilon = 5; K = 5
    n = generate_data(150,M); score = np.zeros(M)
    for i in range(M):
        r0 = int(n[i][0]); r1 = int(n[i][1]); r2 = int(n[i][2])
        s0 = int(n[i][3]); s1 = int(n[i][4]); s2 = int(n[i][5])
        k = [r0,r1,r2,s0,s1,s2]
        score[i] = pseudo_SHD(k)
    t = np.zeros(2)
    for i in range(100):
        s = time.time()
        exp_mec(score, M, K, epsilon)
        e = time.time()
        t[0] += (e-s)
        s = time.time()
        JointPnF(score, M, K, epsilon)
        e = time.time()
        t[1] += (e-s)
    return t/100

In [8]:
# M = 5,000

M = 5000
RT = RunTime(M)

print("Exp.     : ", RT[0] ,"[sec]")
print("Joint PnF: ", RT[1] ,"[sec]")

Exp.     :  0.027379369735717772 [sec]
Joint PnF:  0.007012946605682373 [sec]


In [9]:
# M = 25,000

M = 25000
RT = RunTime(M)

print("Exp.     : ", RT[0] ,"[sec]")
print("Joint PnF: ", RT[1] ,"[sec]")

Exp.     :  0.13664587259292602 [sec]
Joint PnF:  0.03705251693725586 [sec]


In [10]:
# M = 100,000

M = 100000
RT = RunTime(M)

print("Exp.     : ", RT[0] ,"[sec]")
print("Joint PnF: ", RT[1] ,"[sec]")

Exp.     :  0.550368435382843 [sec]
Joint PnF:  0.1864541506767273 [sec]


In [11]:
# M = 500,000

M = 500000
RT = RunTime(M)

print("Exp.     : ", RT[0] ,"[sec]")
print("Joint PnF: ", RT[1] ,"[sec]")

Exp.     :  2.805075373649597 [sec]
Joint PnF:  1.1571199631690978 [sec]


In [12]:
# M = 1,000,000

M = 1000000
RT = RunTime(M)

print("Exp.     : ", RT[0] ,"[sec]")
print("Joint PnF: ", RT[1] ,"[sec]")

Exp.     :  5.650887908935547 [sec]
Joint PnF:  2.4999803566932677 [sec]
